In [2]:
import numpy as np
import pandas as pd 

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_auc_score

In [4]:
train = pd.read_csv("/Users/douzi/Desktop/application_of_data/step1-data/application_train.csv.zip")
test = pd.read_csv("/Users/douzi/Desktop/application_of_data/step1-data/application_test.csv.zip")

In [5]:
def find_missing(data):
    missing_count = data.isnull().sum().values
    total = len(data)
    ratio_missng = missing_count / total * 100
    return pd.DataFrame(data={'column name':data.columns, 'missing_ratio':ratio_missng})

In [6]:
missingcolumns_40 = find_missing(train)
missingcolumns_40_list = list(missingcolumns_40['column name'][missingcolumns_40.missing_ratio>40])

In [7]:
train.drop(missingcolumns_40_list, axis=1, inplace=True)

In [8]:
columns_without_id = [col for col in train.columns if col!='SK_ID_CURR']
#Checking for duplicates in the data.
train[train.duplicated(subset = columns_without_id, keep=False)]
print('The no of duplicates in the data:',train[train.duplicated(subset = columns_without_id, keep=False)]
      .shape[0])

The no of duplicates in the data: 0


In [9]:
le = LabelEncoder()
le_count = 0

# Iterate through the columns
for col in train:
    if train[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(train[col].unique())) <= 2:
            # Train on the training data
            le.fit(train[col])
            # Transform both training and testing data
            train[col] = le.transform(train[col])
            test[col] = le.transform(test[col])
            
            # Keep track of how many columns were label encoded
            le_count += 1
            
# print('%d columns were label encoded.' % le_count)

In [10]:
app_train = pd.get_dummies(train)
app_test = pd.get_dummies(test)

In [11]:
train_labels = app_train['TARGET']

# Align the training and testing data, keep only columns present in both dataframes
app_train, app_test = app_train.align(app_test, join = 'inner', axis = 1)

# Add the target back in
app_train['TARGET'] = train_labels

In [12]:
anom = app_train[app_train['DAYS_EMPLOYED'] == 365243]
non_anom = app_train[app_train['DAYS_EMPLOYED'] != 365243]

In [20]:
# Create an anomalous flag column
app_train['DAYS_EMPLOYED_ANOM'] = app_train["DAYS_EMPLOYED"] == 365243

# Replace the anomalous values with nan
app_train['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)

In [14]:
app_test['DAYS_EMPLOYED_ANOM'] = app_test["DAYS_EMPLOYED"] == 365243
app_train['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)

In [14]:
correlations = app_train.corr()['TARGET'].sort_values()

In [15]:
app_train['DAYS_BIRTH'] = abs(app_train['DAYS_BIRTH'])

In [16]:
age_data = app_train[['TARGET', 'DAYS_BIRTH']]
age_data['YEARS_BIRTH'] = age_data['DAYS_BIRTH'] / 365

# Bin the age data
age_data['YEARS_BINNED'] = pd.cut(age_data['YEARS_BIRTH'], bins = np.linspace(20, 70, num = 11))

In [17]:
age_groups  = age_data.groupby('YEARS_BINNED').mean()

In [18]:
ext_data = app_train[['TARGET','EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH']]
ext_data_corrs = ext_data.corr()